# Getting the Data from Reddit

In [1]:
import pandas as pd
import numpy as np
import requests

import time

In [29]:
params = {
    "subreddit":'datascience',
    "size":1000,
    "user_removed": False, 
    "mod_removed": False,
    #"title:not": ['[removed]', '[deleted]'],
    #"selftext:not": ['[removed]', '[deleted]'],
    "link_flair_text": str('Career')
}

# Base url for submissions
subm_url = 'https://api.pushshift.io/reddit/submission/search'

# Create a request object which submits a get request from the pushshift API
res = requests.get(subm_url, params)

# If statment to protect against bad status codes
if res.status_code != 200:
    print(f"Cannot create datafraame, statuscode = {res.status_code}")
else:
    print(f"Creating dataframe, statuscode = {res.status_code}")
    df = pd.DataFrame(res.json()['data'])
    file_path = f"./{params['subreddit']}.csv"
    df.to_csv(file_path,index = False)
    
time.sleep(20)

Creating dataframe, statuscode = 200


In [28]:
df[df['link_flair_text']=='Career']['link_flair_text']

2      Career
6      Career
11     Career
22     Career
29     Career
        ...  
925    Career
927    Career
929    Career
962    Career
972    Career
Name: link_flair_text, Length: 140, dtype: object

In [113]:
# Function to get back data from a specific endpoint and given parameters

# This can go into our functions.py file

base_url = 'https://api.pushshift.io/reddit/submission/search'
endpoint = 'reddit/submission/search'

before_timestamp = round(time.time())
params = {
        "sort":'desc',
        "subreddit" : 'machinelearning',
        "size" : 1000,
        "user_removed": False, 
        "mod_removed": False,
        "after": None,
        "before": before_timestamp,
        #"title:not": ["[removed]","[deleted]"],
        #"author": "[removed]",
        #"selftext:not": "[removed]","[deleted]"
        #,"link_flair_text"
            }

def subreddit_to_dataframe(endpoint, params):
    # Grab time right now, so we'll get the most recent data
    before_timestamp = round(time.time())
    all_dfs = []
    
    features = ['title','selftext', 'created_utc','num_comments','num_crossposts','score','subreddit']
    
    for _ in range(20):
        print(time.ctime(before_timestamp))
        
        # Reassign the parameter to the new before time stamp
        params['before'] = before_timestamp
        
        # Get Request
        res = requests.get(base_url,params)
        
        if res.status_code == 200:
            print(f'Pulling Data, Status Code: {res.status_code}')
            
            # Json to data
            data = res.json()['data']
            
            oldest_submission_timestamp = data[-1]['created_utc']
            before_timestamp = oldest_submission_timestamp
            
            df = pd.DataFrame(data)[features]
            all_dfs.append(df)
            print(len(all_dfs))
            time.sleep(6)
        else:
            print(f'ERROR CODE FOR RES: {res.status_code}')
            return
            break
    
    # Concatenate list of dataframes
    df_full = pd.concat(all_dfs)
    
    # Create file path to save out
    file_path = f"./{params['subreddit']}.csv"
    
    # Save out
    df_full.to_csv(file_path,index = False)
    return df_full

In [114]:
params = {
        "sort":'desc',
        "subreddit" : 'machinelearning',
        "size" : 1000,
        "user_removed": False, 
        "mod_removed": False,
        "after": None,
        "before": before_timestamp,
        #"title:not": ["[removed]","[deleted]"],
        #"author": "[removed]",
        #"selftext:not": "[removed]","[deleted]"
        #,"link_flair_text"
            }
df = subreddit_to_dataframe(endpoint,params)

Wed Apr 22 13:15:56 2020
Pulling Data, Status Code: 200
1
Sat Apr 11 06:30:00 2020
Pulling Data, Status Code: 200
2
Mon Mar 30 00:24:48 2020
Pulling Data, Status Code: 200
3
Sun Mar 15 16:04:38 2020
Pulling Data, Status Code: 200
4
Sun Mar  1 18:36:31 2020
Pulling Data, Status Code: 200
5
Mon Feb 17 00:27:26 2020
Pulling Data, Status Code: 200
6
Sat Feb  1 02:16:31 2020
Pulling Data, Status Code: 200
7
Fri Jan 17 20:50:44 2020
Pulling Data, Status Code: 200
8
Fri Jan  3 04:21:51 2020
Pulling Data, Status Code: 200
9
Mon Dec 16 06:06:16 2019
Pulling Data, Status Code: 200
10
Sat Nov 30 15:32:12 2019
Pulling Data, Status Code: 200
11
Wed Nov 13 01:59:29 2019
Pulling Data, Status Code: 200
12
Fri Oct 25 04:53:17 2019
Pulling Data, Status Code: 200
13
Wed Oct  9 19:43:10 2019
Pulling Data, Status Code: 200
14
Sun Sep 22 09:36:34 2019
Pulling Data, Status Code: 200
15
Tue Sep  3 17:48:48 2019
Pulling Data, Status Code: 200
16
Sat Aug 17 01:37:19 2019
Pulling Data, Status Code: 200
17
Thu Au

In [95]:
df.head()

,title,selftext,link_flair_text,link_flair_text,created_utc,num_comments,num_crossposts,score,subreddit
0,Bizprospex- Data mining- Data cleansing - Data...,[removed],Projects,Projects,1587535725,0,0,1,datascience
1,[meta] r/datascience what does your Whiteboard...,,NaN,NaN,1587535626,0,0,1,datascience
2,Requesting Resume Critique. I'm a rising senio...,,Discussion,Discussion,1587533931,1,0,1,datascience
3,Boost Your Score in Dell EMC Data Science and ...,[removed],Discussion,Discussion,1587531786,0,0,1,datascience
4,Security in Data Science,I’m a CS student who is currently pursuing his...,Education,Education,1587531479,1,0,1,datascience


In [96]:
df[df['selftext']=='[removed]']

,title,selftext,link_flair_text,link_flair_text,created_utc,num_comments,num_crossposts,score,subreddit
0,Bizprospex- Data mining- Data cleansing - Data...,[removed],Projects,Projects,1587535725,0,0,1,datascience
3,Boost Your Score in Dell EMC Data Science and ...,[removed],Discussion,Discussion,1587531786,0,0,1,datascience
9,"New to all of this, I want to learn",[removed],Discussion,Discussion,1587523883,0,0,1,datascience
12,Topic modeling ? What else ?,[removed],Projects,Projects,1587516804,2,0,1,datascience
14,A silly question: where is the best place to l...,[removed],Discussion,Discussion,1587512560,0,0,1,datascience
...,...,...,...,...,...,...,...,...,...
937,Great Remove Resource of Advanced Pandas Funct...,[removed],Projects,Projects,1583996730,0,0,1,datascience
941,[Academic Survey] Decision making &amp; Data A...,[removed],Meta,Meta,1583990657,0,0,1,datascience
947,Lambda School Data Science Program: Problems a...,[removed],Education,Education,1583973847,0,0,1,datascience
956,Question about a career path as a data scienti...,[removed],Career,Career,1583963926,0,0,1,datascience


In [85]:
time.ctime(1587512560)

'Tue Apr 21 16:42:40 2020'

In [43]:
# Lets merge our titles and selftexts
df['merged'] = df['title'] + df['selftext']

In [58]:
list(df.columns)

['all_awardings',
 'allow_live_comments',
 'author',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_text',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'author_premium',
 'awarders',
 'can_mod_post',
 'contest_mode',
 'created_utc',
 'domain',
 'full_link',
 'gildings',
 'id',
 'is_crosspostable',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'link_flair_background_color',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_template_id',
 'link_flair_text',
 'link_flair_text_color',
 'link_flair_type',
 'locked',
 'media_only',
 'no_follow',
 'num_comments',
 'num_crossposts',
 'over_18',
 'parent_whitelist_status',
 'permalink',
 'pinned',
 'post_hint',
 'preview',
 'pwls',
 'retrieved_on',
 'score',
 'selftext',
 'send_replies',
 'spoiler',
 'stickied',
 'subreddit',
 'subreddit_id',
 'subreddit_subscribers',
 'subreddit_type',
 'thumbnail',
 'title',
 'total_awards

## EDA

In [49]:
list_values = df[['title', 'selftext']].isnull().sum()
list_values

title        0
selftext    31
dtype: int64

In [52]:
#list(df.score)

In [45]:
pd.DataFrame(list_values,index = list_values.index)

,0
all_awardings,0
allow_live_comments,0
author,0
author_flair_css_class,997
author_flair_richtext,51
...,...
author_cakeday,995
distinguished,997
gilded,999
suggested_sort,998


In [46]:
# Create a list of word counts for each post (title + selftext)
list_character_counts = [len(element) for element in df['merged']]

# Create a list of character counts for each post
list_word_counts = [element.count(' ') for element in df['merged']]

TypeError: object of type 'float' has no len()

In [47]:
np.mean(list_character_counts)

253.38

In [23]:
np.mean(list_word_counts)

40.06

In [41]:
df.shape

(50, 63)

## Count Vectorizer

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [29]:
# Instantiate and create parameters for our Count Vectorizer
cvec = CountVectorizer(stop_words = 'english', max_features = 250)
cvec.fit(df['merged']);

# Transform the merged column and save to variable
vectorized_data = cvec.transform(df['merged'])

In [30]:
selftext = pd.DataFrame(vectorized_data.toarray(), columns = cvec.get_feature_names())

In [39]:
selftext.sum()

10              5
8rawdd5zs4w     2
achieve         7
achievements    2
acoustic        2
               ..
year            3
years           3
yoga            4
youtu           6
youtube         2
Length: 250, dtype: int64